In [ ]:
!pip install -U finance-datareader
import pandas as pd
import os
from tqdm import tqdm
import FinanceDataReader as fdr

In [ ]:
통합재무정보 = pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/통합.csv', index_col = 0,  dtype={'종목코드':'str'})

# 데이터 가져옴

In [ ]:
매출액누락 = 통합재무정보[(통합재무정보['매출액'] == '0')|(통합재무정보['매출액'] == 0)]['종목코드'].unique().tolist()
영업이익누락 = 통합재무정보[(통합재무정보['영업이익'] == '0')|(통합재무정보['영업이익'] == 0)]['종목코드'].unique().tolist()
매출총이익누락 = 통합재무정보[(통합재무정보['매출총이익'] == '0')|(통합재무정보['매출총이익'] == 0)]['종목코드'].unique().tolist()
전체누락 = 매출액누락 + 영업이익누락 + 매출총이익누락

In [ ]:
print(f"매출액누락: {len(매출액누락)}")
print(f"영업이익누락: {len(영업이익누락)}")
print(f"매출총이익누락: {len(매출총이익누락)}")

매출액누락: 299
영업이익누락: 222
매출총이익누락: 353


In [ ]:
import glob

재무상태표 = [] # _01_
손익계산서 = [] # _02_
포괄손익계산서 = [] # _03_
현금흐름표 = [] # _04_
자본변동표 = [] #_05_


for name in glob.glob('/content/drive/MyDrive/22캡스톤디자인/재무일괄/*.txt'):
  if '_01_' in name:
    재무상태표.append(name)
  elif '_02_' in name:
    손익계산서.append(name)
  elif '_03_' in name:
    포괄손익계산서.append(name)
  elif '_04_' in name:
    현금흐름표.append(name)
  elif '_05_' in name:
    자본변동표.append(name)

In [ ]:
손익계산샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

for path in tqdm(손익계산서):
  temp = pd.read_csv(path, delimiter = '\t', encoding='cp949')  

  당기 = ''
  if temp['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif temp['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif temp['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif temp['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  temp['사업연도'] = path[52:56] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  temp['항목명'] = temp['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  temp['항목코드'] = temp['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  temp = temp[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]
  temp.rename(columns={당기:"당기"}, inplace=True)

  손익계산샘플 = 손익계산샘플.append(temp)

손익계산샘플['종목코드'] = 손익계산샘플['종목코드'].apply(lambda x:x[1:-1])

 42%|████▏     | 10/24 [00:14<00:09,  1.44it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 24/24 [00:16<00:00,  1.50it/s]


In [ ]:
손익계산샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

for path in tqdm(손익계산서):
  temp = pd.read_csv(path, delimiter = '\t', encoding='cp949')  

  당기 = ''
  if temp['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif temp['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif temp['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif temp['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  temp['사업연도'] = path[52:56] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  temp['항목명'] = temp['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  temp['항목코드'] = temp['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  temp = temp[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]
  temp.rename(columns={당기:"당기"}, inplace=True)

  손익계산샘플 = 손익계산샘플.append(temp)

손익계산샘플['종목코드'] = 손익계산샘플['종목코드'].apply(lambda x:x[1:-1])

100%|██████████| 24/24 [00:02<00:00, 10.42it/s]


# 누락데이터 확인

In [ ]:
누락데이터 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

sample = 손익계산샘플[손익계산샘플['종목코드'].isin(매출액누락 + 영업이익누락 + 매출총이익누락)]
# (수익)매출액 (ifrs-full_Revenue)
매출액 = sample[(sample['항목코드'] == 'ifrs_Revenue') | (sample['항목코드'] == 'ifrs-full_Revenue') | (sample['항목명'].isin(['매출액','(수익)매출액','매출액(수익)','수익(매출액)']))]
매출액.rename(columns={당기:"당기"}, inplace=True)
매출액.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
누락데이터 = 누락데이터.append(매출액, sort=False, ignore_index=True)

 # 매출총이익 (ifrs-full_GrossProfit))
매출총이익 = sample[(sample['항목코드'] == 'ifrs_GrossProfit') | (sample['항목코드'] == 'ifrs-full_GrossProfit')| (sample['항목명'].isin(['매출총이익','(수익)매출총이익','매출총이익(손실)']))]
매출총이익.rename(columns={당기:"당기"}, inplace=True)
매출총이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
누락데이터 = 누락데이터.append(매출총이익, sort=False, ignore_index=True)

# 영업이익(손실) (dart_OperatingIncomeLoss)
영업이익 = sample[(sample['항목코드'] == 'dart_OperatingIncomeLoss')|(sample['항목명'].isin(['영업이익','영업이익(손실)']))]
영업이익.rename(columns={당기:"당기"}, inplace=True)
영업이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
누락데이터 = 누락데이터.append(영업이익, sort=False, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
누락데이터

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,001040,2016,사업보고서,CJ,715,ifrs_Revenue,영업수익,"23,954,197,397,000"
1,000120,2016,사업보고서,CJ대한통운,493,ifrs_Revenue,매출액,"6,081,945,642,204"
2,155660,2016,사업보고서,DSR,139,ifrs_Revenue,매출액,"201,060,062,780"
3,078930,2016,사업보고서,GS,715,ifrs_Revenue,매출,"13,462,426,000,000"
4,006360,2016,사업보고서,GS건설,411,ifrs_Revenue,매출액,"11,035,596,402,664"
...,...,...,...,...,...,...,...,...
11399,005380,2016,3분기보고서,현대자동차,301,dart_OperatingIncomeLoss,영업이익,"4,172,334,000,000"
11400,011760,2016,3분기보고서,현대종합상사,468,dart_OperatingIncomeLoss,영업이익,"23,404,203,949"
11401,008770,2016,3분기보고서,호텔신라,478,dart_OperatingIncomeLoss,V.영업이익,"63,348,832,292"
11402,064240,2016,3분기보고서,홈캐스트,265,dart_OperatingIncomeLoss,영업이익(손실),"17,324,059,573"


---

# 데이터 전처리

In [ ]:
누락데이터['항목명'] = 누락데이터.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs_Revenue' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs-full_Revenue' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs_GrossProfit' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs-full_GrossProfit' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '영업이익' if x['항목코드'] == 'dart_OperatingIncomeLoss' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs_ProfitLoss' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs-full_ProfitLoss' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '감가상각비' if x['항목코드'] == 'dart_DepreciationExpense' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs_ProfitLossBeforeTax' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs-full_ProfitLossBeforeTax' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '매출액' if x['항목명'] == '수익(매출액)' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '매출총이익' if x['항목명'] == '매출총이익(손실)' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '영업이익' if x['항목명'] == '영업이익(손실)' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '당기순이익' if x['항목명'] == '당기순이익(손실)' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세차감전순이익(손실)' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세비용차감전순이익' else x['항목명'], axis=1)

누락데이터['항목코드'] = 누락데이터.apply(lambda x: 'ifrs_Revenue' if x['항목명'] == '매출액' else x['항목코드'], axis=1)
누락데이터['항목코드'] = 누락데이터.apply(lambda x: 'ifrs_GrossProfit' if x['항목명'] == '매출총이익' else x['항목코드'], axis=1)
누락데이터['항목코드'] = 누락데이터.apply(lambda x: 'dart_OperatingIncomeLoss' if x['항목명'] == '영업이익' else x['항목코드'], axis=1)
누락데이터['항목코드'] = 누락데이터.apply(lambda x: 'ifrs_ProfitLoss' if x['항목명'] == '당기순이익' else x['항목코드'], axis=1)
누락데이터['항목코드'] = 누락데이터.apply(lambda x: 'dart_DepreciationExpense' if x['항목명'] == '감가상각비' else x['항목코드'], axis=1)
누락데이터['항목코드'] = 누락데이터.apply(lambda x: 'ifrs-full_ProfitLossBeforeTax' if x['항목명'] == '법인세차감전순이익' else x['항목코드'], axis=1)

누락데이터 =  누락데이터.drop_duplicates()

In [ ]:
누락데이터['항목코드'].unique()

array(['ifrs_Revenue', 'ifrs_GrossProfit', 'dart_OperatingIncomeLoss'],
      dtype=object)

In [ ]:
누락데이터['항목명'].unique()

array(['매출액', '매출총이익', '영업이익'], dtype=object)

In [ ]:
누락데이터

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,001040,2016,사업보고서,CJ,715,ifrs_Revenue,매출액,"23,954,197,397,000"
1,000120,2016,사업보고서,CJ대한통운,493,ifrs_Revenue,매출액,"6,081,945,642,204"
2,155660,2016,사업보고서,DSR,139,ifrs_Revenue,매출액,"201,060,062,780"
3,078930,2016,사업보고서,GS,715,ifrs_Revenue,매출액,"13,462,426,000,000"
4,006360,2016,사업보고서,GS건설,411,ifrs_Revenue,매출액,"11,035,596,402,664"
...,...,...,...,...,...,...,...,...
11399,005380,2016,3분기보고서,현대자동차,301,dart_OperatingIncomeLoss,영업이익,"4,172,334,000,000"
11400,011760,2016,3분기보고서,현대종합상사,468,dart_OperatingIncomeLoss,영업이익,"23,404,203,949"
11401,008770,2016,3분기보고서,호텔신라,478,dart_OperatingIncomeLoss,영업이익,"63,348,832,292"
11402,064240,2016,3분기보고서,홈캐스트,265,dart_OperatingIncomeLoss,영업이익,"17,324,059,573"


# 출력

In [ ]:
누락데이터.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/누락데이터.csv')